# Using GRiTS on simulation data

In [1]:
import numpy as np
import mbuild as mb

from grits import CG_System
from grits.utils import amber_dict

### Coarse-graining

Words, pictures of itic, p3ht
![structures](p3ht-itic.png)

explanation of how grits works and the assumptions it makes when working with gsd files.

In [2]:
itic_backbone = "c1c4c(cc2c1Cc3c2scc3)Cc5c4scc5"
thiophene = "c1cscc1"
a = mb.load(itic_backbone, smiles=True)
a.visualize().show()

b = mb.load(thiophene, smiles=True)
b.visualize().show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [3]:
gsdfile = "../grits/tests/assets/itic-p3ht.gsd"
system = CG_System(
    gsdfile, 
    beads={"_A": itic_backbone, "_B" : thiophene}, 
    conversion_dict=amber_dict
)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is Compound)

/Users/jenny/Projects/grits/grits/coarsegrain.py:150: UserWarning: Some atoms have been left out of coarse-graining!
  warn("Some atoms have been left out of coarse-graining!")


CPU times: user 3.55 s, sys: 68.2 ms, total: 3.62 s
Wall time: 3.95 s


/Users/jenny/Projects/grits/grits/coarsegrain.py:120: UserWarning: c1c4c(cc2c1Cc3c2scc3)Cc5c4scc5 not found in compound!
  warn(f"{smart_str} not found in compound!")


In [4]:
cg_gsd = "itic-p3ht-cg.gsd"
system.save(cg_gsd)

CPU times: user 29.4 ms, sys: 8.79 ms, total: 38.2 ms
Wall time: 59.6 ms


View these in [OVITO](https://www.ovito.org/) or [VMD](https://www.ks.uiuc.edu/Research/vmd/) with [the gsd-vmd plugin](https://github.com/mphowardlab/gsd-vmd).

![itic](cg-itic.png)
![p3ht](cg-p3ht.png)

In [5]:
cg_json = "itic-p3ht-cg.json"
system.save_mapping(cg_json)

Mapping saved to itic-p3ht-cg.json


In [6]:
map_system = CG_System(gsdfile, mapping=cg_json, conversion_dict=amber_dict)

map_cg_gsd = "map-itic-p3ht-cg.gsd"
system.save(map_cg_gsd)

CPU times: user 33.6 ms, sys: 10.5 ms, total: 44.2 ms
Wall time: 57.1 ms


### Fine-graining

WIP